In [ ]:
import os
import numpy as np
import torch
from td3_agent import TD3Agent
import sys
sys.path.append(os.path.abspath('../hockey_env/hockey'))
import hockey_env as h_env
from train_td3 import train_td3

AttributeError: 'TD3Agent' object has no attribute 'select_action'

In [ ]:
# Define model path
model_path = "TD3_Hockey_1000.pth"

# If no saved model exists, train from scratch
if not os.path.exists(model_path):
    print("🚀 No trained model found! Starting training from scratch...")
    train_td3(num_episodes=1000, save_every=100, opponent="weak")  # Options: "weak", "strong", "td3"
else:
    print(f"✅ Found trained model: {model_path}")

In [ ]:
# Initialize Environment (Change opponent type as needed)
env = HockeyEnv_BasicOpponent(weak_opponent=True)  # Choose: weak_opponent=True / False
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
max_action = float(env.action_space.high[0])

# Create TD3 Agent
agent = TD3Agent(state_dim, action_dim, max_action)

# Load trained model if available
if os.path.exists(model_path):
    agent.actor.load_state_dict(torch.load(model_path))  # Load latest trained model
    print(f"📥 Loaded trained model from {model_path}")
else:
    print("🚀 No trained model found, starting fresh!")

In [ ]:
obs, _ = env.reset()
done = False
while not done:
    env.render(mode="human")  # Visualize game
    action = agent.select_action(obs)  # TD3 selects action
    opponent_action = env.opponent.act(env.obs_agent_two())  # AI Opponent
    obs, reward, done, _, _ = env.step(np.hstack([action, opponent_action]))

env.close()  # Close environment after game


In [ ]:
# Train in Shooting Mode
train_td3(num_episodes=500, save_every=50, opponent="weak")  # Train TD3 in shooting mode

In [ ]:
# Train in Defense Mode
train_td3(num_episodes=500, save_every=50, opponent="strong")  # Train TD3 in defense mode

In [ ]:
# Train TD3 vs TD3 (Self-Play)
train_td3(num_episodes=1000, save_every=100, opponent="td3")  # TD3 vs TD3 self-play training